In [6]:
#Import dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs

In [2]:
#Define the path to the chrome driver
def init_browser():
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, headless=True)

In [3]:
#Define the scrape function

browser = init_browser()

This section allows for the scraping of the Mars Facts table
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

html = browser.html
soup = bs(html, "html.parser")

mars_table = soup.find('', class_ = 'tablepress-p-mars-no-2')

browser.quit()

    #return listings

In [9]:
url = "https://space-facts.com/mars/"
tables = pd.read_html(url)
tables[0]

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [ ]:
def scrape():  
    
    #Define a dictionary called scraped_data that will be returned by the function
    
    #Open a browser window
    browser = init_browser()
    
    #This section allows for the scraping of NASA's Mars Exploration website
    #Feed the URL and visit the website
    url = "https://mars.nasa.gov/news/"
    browser.visit(url)

    #Obtain the HTML and parse it into a BeautifulSoup object.
    html = browser.html
    soup = bs(html, "html.parser")

    #Use the find function to find the most recent news title and description.
    news_title = soup.find('', class_ = 'list_text').find('a').get_text()
    news_p = soup.find('', class_ = 'article_teaser_body').get_text()
    
    #This section allows for the scraping of the JPL Featured Space Image website
    url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    browser.visit(url)

    html = browser.html
    soup = bs(html, "html.parser")

    featured_image_tag = soup.find('', class_ = 'button fancybox')['data-fancybox-href']
    featured_image_url = f'https://www.jpl.nasa.gov/{featured_image_tag}'
    
    #This section allows for the scraping of the Mars Facts table
    
    
    #This section allows for the scraping of ...
    
    browser.quit()
    
    

In [ ]:
scrape()